In [1]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:00


In [ ]:
#For compatiblaity of audio model
!pip uninstall transformers
!pip install transformers==4.38.2

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM



MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

model.eval()




def chat(prompt, system_prompt="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]


    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        text,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.7,
            top_p=0.93,
            do_sample=True,
            repetition_penalty=1.2
        )

    response = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[-1]:],
        skip_special_tokens=True
    )

    return response.strip()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from typing import List, Tuple
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from PIL import Image, ImageDraw, ImageFont
import re


def divide_text(text):
    return [word for word in text.split(" ") if word]


def build_pdf(
    qa_pairs: List[Tuple],
    output_path="output.pdf",
    page_bg_path="image.png",
    font_path="font.ttf",
):

    NEW_W = 1024
    NEW_H = 1536

    FONT_SIZE = 57
    SPACE = 22
    LEFT_MARGIN = 145
    TOP_MARGIN = 205
    LINE_HEIGHT = 48
    ANSWER_GAP = 94
    TAB_SPACE = 62
    ARROW_X = 67
    PAGE_END = 1004

    c = Canvas(output_path, A4)
    pdf_w, pdf_h = A4

    font = ImageFont.truetype(font_path, size=FONT_SIZE)

    for q, a in qa_pairs:

        img = Image.open(page_bg_path).resize((NEW_W, NEW_H))
        draw = ImageDraw.Draw(img)

        q_parts = divide_text(q)
        a_parts = divide_text(a)

        X, Y = LEFT_MARGIN, TOP_MARGIN
        for word in q_parts:
            bbox = draw.textbbox((0, 0), word, font=font)
            word_width = bbox[2] - bbox[0]

            if X + word_width > PAGE_END:
                X = LEFT_MARGIN
                Y += LINE_HEIGHT

            draw.text((X, Y), word, fill=(170, 0, 0), font=font)
            X += word_width + SPACE

        Y += ANSWER_GAP
        draw.text((ARROW_X, Y), "->", fill=(40, 40, 40), font=font)

        X = LEFT_MARGIN

        for word in a_parts:

            if word == "</n>":
                Y += LINE_HEIGHT
                X = LEFT_MARGIN
                continue

            if word == "</t>":
                X += TAB_SPACE
                continue




            bbox = draw.textbbox((0, 0), word, font=font)
            word_width = bbox[2] - bbox[0]





            if X + word_width > PAGE_END:
                X = LEFT_MARGIN
                Y += LINE_HEIGHT

            if Y + (bbox[3] - bbox[1]) + TOP_MARGIN / 3 > NEW_H:
              img_data = ImageReader(img)
              c.drawImage(img_data, 0, 0, pdf_w, pdf_h)
              c.showPage()
              img = Image.open(page_bg_path).resize((NEW_W, NEW_H))
              draw = ImageDraw.Draw(img)
              X, Y = LEFT_MARGIN, TOP_MARGIN

            if word == "</c>":
                X += int((pdf_w - word_width * 1.5) / 2)
                continue

            draw.text((X, Y), word, fill=(15, 15, 15), font=font)
            X += word_width + SPACE

        img_data = ImageReader(img)
        c.drawImage(img_data, 0, 0, pdf_w, pdf_h)
        c.showPage()

    c.save()

In [ ]:
def get_questions(text: str):
  return text.split("|")

def get_questions_with_answers(questions):
  all = []
  for question in questions:
    answer = clean_latex(chat(f"{question} With main title at beggining in short."))
    all.append((question, answer))

  return all



def questions_to_pdf(text: str):
  """
  Takes text and creates a pdf with all its answers.
  The text must have questions seperated by | symbol to work.
  """

  questions = get_questions(text)
  structured_parts = get_questions_with_answers(questions)
  build_pdf(structured_parts, page_bg_path="pdf_page.jpeg")



In [ ]:
import re

def clean_latex(text: str) -> str:
    """
    Converts common LaTeX math into readable plain text
    without corrupting fractions, powers, subscripts, etc.
    Preserves newlines and custom formatting tags.
    """

    # ─────────────────────────────────────────────
    # Remove math mode wrappers
    # ─────────────────────────────────────────────
    text = re.sub(r"\\\[(.*?)\\\]", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"\\\((.*?)\\\)", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"\$\$(.*?)\$\$", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"\$(.*?)\$", r"\1", text)

    # ─────────────────────────────────────────────
    # Handle fractions FIRST (recursive-safe simple version)
    # \frac{a}{b} → (a/b)
    # ─────────────────────────────────────────────
    while re.search(r"\\frac\{([^{}]+)\}\{([^{}]+)\}", text):
        text = re.sub(
            r"\\frac\{([^{}]+)\}\{([^{}]+)\}",
            r"(\1/\2)",
            text
        )

    # ─────────────────────────────────────────────
    # Superscripts
    # x^{2} → x^2
    # x^2   → x^2
    # ─────────────────────────────────────────────
    text = re.sub(r"\^\{([^{}]+)\}", r"^\1", text)

    # ─────────────────────────────────────────────
    # Subscripts
    # x_{1} → x1
    # x_1   → x1
    # ─────────────────────────────────────────────
    text = re.sub(r"_\{([^{}]+)\}", r"\1", text)
    text = re.sub(r"_([0-9a-zA-Z]+)", r"\1", text)

    # ─────────────────────────────────────────────
    # Common symbol replacements
    # ─────────────────────────────────────────────
    replacements = {
        r"\propto": " proportional to ",
        r"\approx": " approximately equal to ",
        r"\neq": " not equal to ",
        r"\geq": " greater than or equal to ",
        r"\leq": " less than or equal to ",
        r"\cdot": " * ",
        r"\times": " * ",
        r"\div": " / ",
        r"\pm": " ± ",
        r"\sqrt": " sqrt ",
    }

    for latex, word in replacements.items():
        text = text.replace(latex, word)

    # ─────────────────────────────────────────────
    # Remove remaining unknown LaTeX commands
    # but keep their arguments
    # Example: \text{hello} → hello
    # ─────────────────────────────────────────────
    text = re.sub(r"\\[a-zA-Z]+\{([^{}]*)\}", r"\1", text)

    # Remove standalone commands like \alpha (optional: you can map these if needed)
    text = re.sub(r"\\[a-zA-Z]+", "", text)

    # Remove leftover braces
    text = text.replace("{", "").replace("}", "")

    # ─────────────────────────────────────────────
    # Formatting markers
    # ─────────────────────────────────────────────
    text = re.sub(r"[ \t]+", " ", text)
    text = text.replace("\n", " </n> ")
    text = text.replace("\t", " </t> ")
    text = text.replace("###", " </c> ")
    text = text.replace("#", " </c> ")
    text = text.replace("***", "")
    text = text.replace("**", "")

    return text.strip()

In [ ]:
text = "Solve: x / 3 + 2x = 10 - x no explanation at all but every intermediate equations. Also use / for division symbol. "
questions_to_pdf(text)

### With Audio model.


In [ ]:
from transformers import pipeline

In [ ]:
audio_pipe = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-tiny",
    device = "cuda"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json: 0.00B [00:00, ?B/s]

In [ ]:
import pydub
def audio_to_pdf(audiopath):
  TEMP = "temp.mp3"
  audio = pydub.AudioSegment.from_file(audiopath)
  audio.export(TEMP)
  text = audio_pipe(TEMP)["text"]
  text = text.replace("?", "?|")
  if text[-1]=="|":
    text = text[:-2]

  questions_to_pdf(text)


In [ ]:
audio_to_pdf("try.m4a")